# RespGroup_logTTP: conventional NN models

In [ ]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#Importing to preprocess the data
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.preprocessing import label_binarize
#Importing to build the models
from tensorflow.keras import layers, regularizers, models
#Importing to evaluate the models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
#Importing to explain the models
import shap

# for reproducibility, the value is set for conventional reasons
SEED = 42
tf.keras.utils.set_random_seed(SEED)

In [ ]:
# load the data
data = pd.read_csv('dataset_d.csv', encoding='latin-1', sep=',') #request the dataset to the author
#data.head()

In [ ]:
# target column : "progression_log_cat" multi-classification problem
# relevant columns for the model
relevant_columns = [ 'age', 'sex', 'smoking', 'ps_at_diagnosis_ad', 'n#_mets_sites', 'lung_only_m1', 'pleural', 'pericard','lymph_nodes_only_m1','soft_tissue',
    'leptomingeal','skin','peritoneal','renal','pancreas', 'brain', 'liver', 'bone', 'adrenal', 'histology', 'hbbaselineio','leucotbaselineio',
    'neut_abs...143','linfo_abs...144','baso_abs...145', 'mono_abs...147', 'plaqtbaselineio', 'progression_log_cat']
df= data.copy()
df = df[relevant_columns]
df.shape

In [ ]:
df = df.dropna(axis=0)
df.shape

In [ ]:
#to randomize the data
df = df.sample(frac=1, random_state=SEED)

var_int = ['ps_at_diagnosis_ad', 'n#_mets_sites', 'lung_only_m1', 'pleural', 'pericard', 'lymph_nodes_only_m1', 'soft_tissue',
           'leptomingeal','skin','peritoneal','renal','pancreas', 'brain', 'liver', 'bone', 'adrenal']
for i in var_int:
    df[i] = df[i].astype(int)
    
df['sex'] = df['sex'].str.lower()

sex_dummies = pd.get_dummies(df['sex'], prefix='sex', drop_first=True)

other_dummies = pd.get_dummies(df[['histology', 'smoking']])

df_encoded = pd.concat([df.drop(columns=['sex', 'histology', 'smoking']),
                        sex_dummies, other_dummies], axis=1)

cols_to_convert = ['histology_adenocarcinoma', 'histology_nsclc', 'histology_squamous' ,'sex_male', 
                   'smoking_current', 'smoking_former', 'smoking_non-smoker'] #,'histology_adenosquamous'

df_encoded[cols_to_convert] = df_encoded[cols_to_convert].astype(int)

In [ ]:
# split the data into features and target
X = df_encoded[df_encoded.columns.difference(['progression_cat'])]  # Exclude the target column
y = df_encoded['progression_cat']  

le = LabelEncoder()
y_encoded = le.fit_transform(y) 

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

print("Shape of X_train_val:",X_train_val.shape)
print("Shape of X_train:",X_train.shape)
print("Shape of X_val:",X_val.shape)
print("Shape of X_test:",X_test.shape)

In [ ]:
binary_features = ['lung_only_m1', 'pleural', 'pericard', 'lymph_nodes_only_m1', 'soft_tissue', 'leptomingeal','skin','peritoneal','renal',
                   'pancreas', 'brain', 'liver', 'bone', 'adrenal','histology_adenocarcinoma', 'histology_nsclc', 
                   'histology_squamous', 'sex_male','smoking_current', 'smoking_former', 'smoking_non-smoker' ]#'histology_adenosquamous',
numeric_features = ['neut_abs...143','linfo_abs...144', 'plaqtbaselineio', 'age', 'ps_at_diagnosis_ad', 'n#_mets_sites', 'leucotbaselineio',
                    'hbbaselineio','baso_abs...145', 'mono_abs...147'] #'duration_l1', 'time_to_l1_start'


X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy() 
X_test_scaled = X_test.copy()
X_train_val_scaled = X_train_val.copy()

scaler = StandardScaler()

X_train_val_scaled[numeric_features] = scaler.fit_transform(X_train_val_scaled[numeric_features])
X_train_scaled[numeric_features] = scaler.fit_transform(X_train_scaled[numeric_features])
X_val_scaled[numeric_features] = scaler.fit_transform(X_val_scaled[numeric_features])
X_test_scaled[numeric_features] = scaler.fit_transform(X_test_scaled[numeric_features])

In [ ]:
for idx, label in enumerate(le.classes_):
    print(f"{idx}: {label}")


In [ ]:
# model with 1 hidden layer with 512 neurons and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)


In [ ]:
X_scaled = X.copy()
X_scaled[numeric_features] = scaler.fit_transform(X_scaled[numeric_features])

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_train_val_scaled_df = pd.DataFrame(X_train_val_scaled, columns=X_train_val.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False 
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 1 hidden layer with 512 neurons, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False 
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 1 hidden layer with 512 neurons, Norm layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.LayerNormalization(),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False  
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 1 hidden layer with 512 neurons, Batch layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])



# compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# train the model
history = mlp_model.fit(X_train_scaled, y_train,
              validation_data=(X_val_scaled, y_val),
              epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False  
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 2 hidden layers, each of them with 512 neurons, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False  
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 2 hidden layers, each of them with 512 neurons, Norm layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)), 
    layers.LayerNormalization(center=True, scale=True),
    layers.Dropout(0.4),  
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=X_test_scaled_df.columns,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False  
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()

In [ ]:
# model with 2 hidden layers, each of them with 512 neurons, Norm layer, ElasticNet and dropout 0.2
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)), 
    layers.LayerNormalization(center=True, scale=True),
    layers.Dropout(0.2),  
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)),
    layers.Dropout(0.2),
    layers.Dense(3, activation='softmax')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
class_predictions = np.argmax(prob_predictions, axis=1)

original_predictions = le.inverse_transform(class_predictions)

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions, average='weighted')
recall = recall_score(y_test, class_predictions, average='weighted')
f1 = f1_score(y_test, class_predictions, average='weighted')

y_test_bin = label_binarize(y_test, classes=[0,1,2])
roc_auc = roc_auc_score(y_test_bin, prob_predictions, multi_class='ovr', average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

print(classification_report(y_test, class_predictions))
print(confusion_matrix(y_test, class_predictions))

prob_predictions = mlp_model.predict(X_scaled)
class_predictions = np.argmax(prob_predictions, axis=1)
original_predictions = le.inverse_transform(class_predictions)
df['Predicted'] = original_predictions

print(df.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)
shap_values = explainer.shap_values(X_test_scaled_df, nsamples=100, silent=True)

class_names = le.classes_  
for i, class_name in enumerate(class_names):
    shap.summary_plot(
        shap_values[:,:,i], 
        features=X_test_scaled_df,
        feature_names=binary_features + numeric_features,
        plot_type='dot',
        max_display=len(binary_features + numeric_features),
        show=False  
    )
    plt.title(f"Class {class_name}")
    plt.savefig(f"shap_plot_class_{class_name}.png")
    plt.close()


fig, axs = plt.subplots(1, 3, figsize=(16, 12))
axs = axs.flatten()

for i, class_name in enumerate(class_names):
    img = plt.imread(f"shap_plot_class_{class_name}.png")
    axs[i].imshow(img)
    axs[i].axis('off')
    axs[i].set_title(f"Class {class_name}")

plt.tight_layout()
plt.show()